In [1]:
import os
import sys
import subprocess

from from_root import from_root
from datetime import datetime

sys.path.insert(0, str(from_root("src")))

from utils import get_base_location, apply_temp_doc_id
from read_and_write_docs import read_jsonl

In [2]:

# To the minute (seconds set to "00")
now = datetime.now()
stamp_seconds = now.strftime("%Y%m%d%H%M%S")  # e.g. "20251008142357"

corpus      = "Wiki"
data_type   = "test"

# Set NAS so can run on Windows laptop seamlessly
nas_base_loc = get_base_location()

known_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
unknown_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
metadata_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/metadata.rds"
model_loc = f"{nas_base_loc}/models/Qwen 2.5/Qwen2.5-0.5B-Instruct"
save_loc = f"{nas_base_loc}/paraphrase examples/{corpus}-{data_type}"
completed_loc = f"{nas_base_loc}/paraphrase examples/{corpus}-{data_type}-completed"

# known_loc = "/Users/user/Documents/test_data/Wiki/known_raw.jsonl"
# unknown_loc = "/Users/user/Documents/test_data/Wiki/unknown_raw.jsonl"
# metadata_loc = "/Users/user/Documents/test_data/metadata.rds"
# model_loc = "/Users/user/Documents/models/Qwen2.5-0.5B-Instruct"

paraphrase_model = "Qwen-2.5-0.5B"
max_tokens   = 5000
temperature  = 0.7
n_samples    = 10

model_str = paraphrase_model.replace(".", "-").replace("_", "-")

save_loc = f"{nas_base_loc}/paraphrase examples/{corpus}-{data_type}-{model_str}"
completed_loc = f"{nas_base_loc}/paraphrase examples/{corpus}-{data_type}-{model_str}-completed"
print(f"Saving to: {save_loc}")

script_loc = str(from_root("scripts", "run_hf_paraphrase_method.py"))

Saving to: /Volumes/BCross/paraphrase examples/Wiki-test-Qwen-2-5-0-5B


In [3]:
known = read_jsonl(known_loc)
known = apply_temp_doc_id(known)

In [4]:
print(known[known['doc_id'] == 'hootmag_text_10'].reset_index().loc[0, 'text'])

Hello Halidshou, I think you didn't have to make this edit, that omits San Marino from IMF list of advanced economies, beacsue the original text that had San Marino on the list gave a clear source for that.
Anyways, yesterday IMF published its new report, in which San Marino was included again on the list, see ibid.
If you do, then please tell me if you think that every model - uniquely determined up to isomorphism by an 'infinite' class of second order axioms, can also be uniquely determined up to isomorphism by a 'finite' class of second order axioms?
Additionally, can every first order proposition having a model, be contained in a finite class - of first/second order propositions - that uniquely determines a model up to isomorphism ?
Anyways, my previous response to you seemed to be a riddle - just because you asked two questions but while you were looking forward to a one-word answer, which is impossible, of course.


In [5]:
# Load the pre-made problem datasets for speed
problem_dataset_base = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}"
problem_dataset_agg = read_jsonl(f"{problem_dataset_base}/{corpus}_{data_type}_agg.jsonl")
# problem_dataset_agg = read_jsonl("/Users/user/Documents/test_data/Wiki/Wiki_test_agg.jsonl")

In [6]:
same_probs = problem_dataset_agg[problem_dataset_agg['known_author'] == problem_dataset_agg['unknown_author']].copy()
same_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
same_probs[(same_probs['highest_common_count'] >= 3) & (same_probs['highest_common_count'] <= 10)].head(10)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
601,Vedant vs Vedant,Vedant,Vedant,vedant_text_2,vedant_text_4,10,this Ġis Ġthe Ġproblem Ġwhen Ġindian Ġnational...
236,Mayalld vs Mayalld,Mayalld,Mayalld,mayalld_text_4,mayalld_text_3,10,"Ġunder Ġthe Ġname Ġof Ġthe Ġmaster Ġaccount , ..."
546,Swift&silent vs Swift&silent,Swift&silent,Swift&silent,swift_silent_text_1,swift_silent_text_2,10,"ug ab oy 5 3 5 1 3 6 ,"
506,Snowded vs Snowded,Snowded,Snowded,snowded_text_2,snowded_text_1,8,Ġis Ġa Ġbehavioural Ġissue Ġnot Ġa Ġcontent Ġone
138,KBlott vs KBlott,KBlott,KBlott,kblott_text_3,kblott_text_1,8,Ġevidence Ġto Ġsupport Ġthe Ġpractice Ġof Ġwit...
426,Richard_Daft vs Richard_Daft,Richard_Daft,Richard_Daft,richard_daft_text_2,richard_daft_text_4,7,Ġpre Ġ 1 8 0 0 Ġcricket
588,U21980 vs U21980,U21980,U21980,u21980_text_1,u21980_text_5,7,Ġnx iv m Ġand Ġr ani ere
462,Scheinwerfermann vs Scheinwerfermann,Scheinwerfermann,Scheinwerfermann,scheinwerfermann_text_11,scheinwerfermann_text_10,7,Ġto Ġfind Ġproduction Ġstart Ġand Ġend Ġdates
565,Thekohser vs Thekohser,Thekohser,Thekohser,thekohser_text_10,thekohser_text_11,7,"Ġ 1 6 , 0 0 0"
661,Yoenit vs Yoenit,Yoenit,Yoenit,yoenit_text_4,yoenit_text_2,7,. Ġ' y oen it ' Ġ


In [7]:
#diff_probs = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
#diff_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
#diff_probs[(diff_probs['highest_common_count'] >= 3) & (diff_probs['highest_common_count'] <= 10)].head(30)

In [8]:
def build_test_dict(df, problem, start=1):
    """
    Build a dictionary of tests from a dataframe with known/unknown doc IDs.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns ['problem', 'known_doc_id', 'unknown_doc_id']
    problem : str
        Problem name to filter on
    start : int, optional
        Starting number for test IDs (default is 1)

    Returns
    -------
    dict
        Dictionary of tests in the form:
        {
            "test_01": {"known": ..., "unknown": ...},
            "test_02": {"known": ..., "unknown": ...},
            ...
        }
    """
    # Filter rows for the given problem
    sub_df = df[df['problem'] == problem]

    # Build dictionary
    return {
        f"test_{i:02d}": {"known": row.known_doc_id, "unknown": row.unknown_doc_id}
        for i, row in enumerate(sub_df.itertuples(index=False), start)
    }

In [9]:
# build_test_dict(same_probs, 'Lear_21 vs Lear_21', 40)

In [10]:
tests = {
    "test_01": {"known": "vedant_text_5", "unknown": "vedant_text_4"},
    "test_02": {"known": "vedant_text_2", "unknown": "vedant_text_4"},
    "test_03": {"known": "vedant_text_1", "unknown": "vedant_text_4"},
    "test_04": {"known": "mayalld_text_4", "unknown": "mayalld_text_3"},
    "test_05": {"known": "mayalld_text_1", "unknown": "mayalld_text_3"},
    "test_06": {"known": "mayalld_text_2", "unknown": "mayalld_text_3"},
    "test_07": {"known": "swift_silent_text_1", "unknown": "swift_silent_text_2"},
    "test_08": {"known": "swift_silent_text_5", "unknown": "swift_silent_text_2"},
    "test_09": {"known": "swift_silent_text_4", "unknown": "swift_silent_text_2"},
    "test_10": {"known": "snowded_text_2", "unknown": "snowded_text_1"},
    "test_11": {"known": "snowded_text_12", "unknown": "snowded_text_1"},
    "test_12": {"known": "snowded_text_10", "unknown": "snowded_text_1"},
    'test_13': {'known': 'kblott_text_4', 'unknown': 'kblott_text_1'},
    'test_14': {'known': 'kblott_text_3', 'unknown': 'kblott_text_1'},
    'test_15': {'known': 'kblott_text_5', 'unknown': 'kblott_text_1'},
    'test_16': {'known': 'richard_daft_text_5', 'unknown': 'richard_daft_text_4'},
    'test_17': {'known': 'richard_daft_text_2', 'unknown': 'richard_daft_text_4'},
    'test_18': {'known': 'richard_daft_text_3', 'unknown': 'richard_daft_text_4'}, 
    'test_19': {'known': 'u21980_text_1', 'unknown': 'u21980_text_5'},
    'test_20': {'known': 'u21980_text_4', 'unknown': 'u21980_text_5'},
    'test_21': {'known': 'u21980_text_3', 'unknown': 'u21980_text_5'},
    'test_22': {'known': 'scheinwerfermann_text_11', 'unknown': 'scheinwerfermann_text_10'},
    'test_23': {'known': 'scheinwerfermann_text_13', 'unknown': 'scheinwerfermann_text_10'},
    'test_24': {'known': 'scheinwerfermann_text_12', 'unknown': 'scheinwerfermann_text_10'},
    'test_25': {'known': 'thekohser_text_10', 'unknown': 'thekohser_text_11'},
    'test_26': {'known': 'thekohser_text_1', 'unknown': 'thekohser_text_11'},
    'test_27': {'known': 'thekohser_text_3', 'unknown': 'thekohser_text_11'},
    'test_28': {'known': 'yoenit_text_4', 'unknown': 'yoenit_text_2'},
    'test_29': {'known': 'yoenit_text_5', 'unknown': 'yoenit_text_2'},
    'test_30': {'known': 'yoenit_text_1', 'unknown': 'yoenit_text_2'},
    'test_31': {'known': 'hootmag_text_12', 'unknown': 'hootmag_text_13'},
    'test_32': {'known': 'hootmag_text_10', 'unknown': 'hootmag_text_13'},
    'test_33': {'known': 'hootmag_text_1', 'unknown': 'hootmag_text_13'},
    'test_34': {'known': 'icarus3_text_3', 'unknown': 'icarus3_text_4'},
    'test_35': {'known': 'icarus3_text_2', 'unknown': 'icarus3_text_4'},
    'test_36': {'known': 'icarus3_text_1', 'unknown': 'icarus3_text_4'},
    'test_37': {'known': 'rjecina_text_10', 'unknown': 'rjecina_text_11'},
    'test_38': {'known': 'rjecina_text_13', 'unknown': 'rjecina_text_11'},
    'test_39': {'known': 'rjecina_text_1', 'unknown': 'rjecina_text_11'},
    'test_40': {'known': 'lear_21_text_2', 'unknown': 'lear_21_text_3'},
    'test_41': {'known': 'lear_21_text_1', 'unknown': 'lear_21_text_3'},
    'test_42': {'known': 'lear_21_text_5', 'unknown': 'lear_21_text_3'}
}

In [11]:
# sys.executable.replace("c:\\", "C:/").replace("\\", "/")

In [12]:
# script_loc.replace("C:\\", "C:/").replace("\\", "/")

## Test single case

In [13]:
# test_num = "test_32"
# entry = tests[test_num]

# known_doc = entry['known']
# unknown_doc = entry['unknown']

# print(f"Working on {test_num}: {known_doc} vs {unknown_doc}")

# fake_save_loc = "/Users/user/Documents/test_data/Wiki/test-docs"
# fake_completed_loc = "/Users/user/Documents/test_data/Wiki/test-docs-completed"

# env = dict(os.environ, PYTHONUNBUFFERED="1")

# cmd = [
#     sys.executable.replace("c:\\", "C:/").replace("\\", "/"), "-u",
#     script_loc.replace("C:\\", "C:/").replace("\\", "/"),
#     "--known_loc", known_loc,
#     "--unknown_loc", unknown_loc,
#     "--metadata_loc", metadata_loc,
#     "--model_loc", model_loc,
#     "--prompt_loc", "/Users/user/Documents/GitHub/av_distributions/prompts/exhaustive_constrained_ngram_paraphraser_prompt_JSON_newv2.txt",
#     "--save_loc", fake_save_loc,
#     "--completed_loc", fake_completed_loc,
#     "--corpus", corpus,
#     "--data_type", data_type,
#     "--known_doc", known_doc,
#     "--unknown_doc", unknown_doc,
#     "--openai_model", openai_model,
#     "--max_tokens", str(max_tokens),
#     "--temperature", str(temperature),
#     "--n", str(n_samples),
#     "--score_texts"
# ]

# print("Running command:\n", " ".join(cmd), "\n")

# process = subprocess.Popen(
#     cmd,
#     stdout=subprocess.PIPE,
#     stderr=subprocess.STDOUT,
#     text=True,
#     env=env,
#     bufsize=1
# )

# # Live output
# try:
#     for line in process.stdout:
#         print(line, end="")  # stream to notebook/console

#     process.stdout.close()
#     return_code = process.wait()

#     if return_code != 0:
#         print(f"\n❌ Process exited with code {return_code}")
#     else:
#         print("\n✅ Process completed successfully")

# except Exception as e:
#     print(f"\n💥 Exception occurred: {e}")
#     process.kill()

In [14]:
# m = re.search(r'<<<RESULT_JSON_START>>>\s*(\{.*\})\s*<<<RESULT_JSON_END>>>', result.stdout, re.S)
# result_dict = json.loads(m.group(1))
# result_dict

In [15]:
tests = {
    'test_31': {'known': 'hootmag_text_12', 'unknown': 'hootmag_text_13'},
}

In [16]:
env = dict(os.environ, PYTHONUNBUFFERED="1")

for test_num, entry in tests.items():

    known_doc = entry['known']
    unknown_doc = entry['unknown']
    print(f"Working on {test_num}: {known_doc} vs {unknown_doc}")
    
    cmd = [
        sys.executable.replace("c:\\", "C:/").replace("\\", "/"), "-u", script_loc.replace("C:\\", "C:/").replace("\\", "/"),
        "--known_loc", known_loc,
        "--unknown_loc", unknown_loc,
        "--metadata_loc", metadata_loc,
        "--model_loc", model_loc,
        "--paraphrase_model_loc", model_loc,
        "--save_loc", save_loc,
        "--prompt_loc", "/Users/user/Documents/GitHub/av_distributions/prompts/exhaustive_constrained_ngram_paraphraser_prompt_JSON_newv2.txt",
        "--completed_loc", completed_loc,
        "--corpus", corpus,
        "--data_type", data_type,
        "--known_doc", known_doc,
        "--unknown_doc", unknown_doc,
        "--max_tokens", str(max_tokens),
        "--temperature", str(temperature),
        "--n", str(n_samples),
        "--score_texts"
    ]

    subprocess.run(cmd, text=True, env=env, check=True)

Working on test_31: hootmag_text_12 vs hootmag_text_13
Working on problem: hootmag_text_12 vs hootmag_text_13
Loading tokenizer/model for n-gram scoring
Loading local HF model for paraphrasing
Loading data
Data loaded
Getting common n-grams
There are 3 n-grams in common!
Generating paraphrases with local HF model


KeyboardInterrupt: 